In [406]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from datetime import time
import matplotlib.pyplot as pplot
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import math
from sklearn.metrics import mean_squared_error
from math import sqrt

In [286]:
pd_volume_train = pd.read_csv('processed_train_volume2.csv')
pd_volume_train.tail()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type,weekday,weekend,hour,date,holiday,time_window
543694,2016-10-17 18:18:22,1,1,1,1,0,1,0,18,2016-10-17,False,"[18:00:00,18:20:00)"
543695,2016-10-17 18:18:49,1,1,1,1,0,1,0,18,2016-10-17,False,"[18:00:00,18:20:00)"
543696,2016-10-17 18:20:03,1,1,1,1,0,1,0,18,2016-10-17,False,"[18:20:00,18:40:00)"
543697,2016-10-17 17:20:20,1,1,1,1,0,1,0,17,2016-10-17,False,"[17:20:00,17:40:00)"
543698,2016-10-17 17:21:52,1,1,1,1,0,1,0,17,2016-10-17,False,"[17:20:00,17:40:00)"


In [275]:
#Função que será usada para obter a janela de tempo de 5 minutos
def get_timewindow(t):
        time_window = 5
        if t.minute < time_window:
            window = [time(t.hour, 0), time(t.hour,5)]
        elif t.minute < time_window*2:
            window = [time(t.hour, 5), time(t.hour, 10)]
        elif t.minute < time_window*3:
            window = [time(t.hour, 10), time(t.hour, 15)]
        elif t.minute < time_window*4:
            window = [time(t.hour, 15), time(t.hour, 20)]
        elif t.minute < time_window*5:
            window = [time(t.hour, 20), time(t.hour, 25)]
        elif t.minute < time_window*6:
            window = [time(t.hour, 25), time(t.hour, 30)]
        elif t.minute < time_window*7:
            window = [time(t.hour, 30), time(t.hour, 35)]
        elif t.minute < time_window*8:
            window = [time(t.hour, 35), time(t.hour, 40)]
        elif t.minute < time_window*9:
            window = [time(t.hour, 40), time(t.hour, 45)]
        elif t.minute < time_window*10:
            window = [time(t.hour, 45), time(t.hour, 50)]
        elif t.minute < time_window*11:
            window = [time(t.hour, 50), time(t.hour, 55)]
        else:
            try:
                window = [time(t.hour, 55), time(t.hour + 1, 0)]
            except ValueError:
                window = [time(t.hour, 55), time(0,0,0)]
        s_window = '[' + str(window[0]) + ',' + str(window[1]) + ')'
        return s_window

def get_hour(t):
        return t.hour

In [124]:
#Função que será usada para obter a janela de tempo de 20 minutos
def get_timewindow20(t):
        time_window = 20
        if t.minute < time_window:
            window = [time(t.hour, 0), time(t.hour,20)]
        elif t.minute < time_window*2:
            window = [time(t.hour, 20), time(t.hour, 40)]
        else:
            try:
                window = [time(t.hour, 40), time(t.hour + 1, 0)]
            except ValueError:
                window = [time(t.hour, 40), time(0,0,0)]
        s_window = '[' + str(window[0]) + ',' + str(window[1]) + ')'
        return s_window

def get_hour(t):
        return t.hour

In [287]:
del pd_volume_train['time_window']
pd_volume_train['time'] = pd.to_datetime(pd_volume_train['time'], format = '%Y-%m-%d %H:%M:%S')
pd_volume_train['t'] = pd_volume_train['time'].dt.time
pd_volume_train['time_window'] = pd_volume_train['t'].apply(get_timewindow)
#del pd_volume_train['t']
pd_volume_train.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type,weekday,weekend,hour,date,holiday,t,time_window
0,2016-09-19 23:09:25,2,0,1,0,0,1,0,23,2016-09-19,False,23:09:25,"[23:05:00,23:10:00)"
1,2016-09-19 23:11:53,2,0,1,0,0,1,0,23,2016-09-19,False,23:11:53,"[23:10:00,23:15:00)"
2,2016-09-19 23:13:54,2,0,1,0,0,1,0,23,2016-09-19,False,23:13:54,"[23:10:00,23:15:00)"
3,2016-09-19 23:17:48,1,0,1,1,0,1,0,23,2016-09-19,False,23:17:48,"[23:15:00,23:20:00)"
4,2016-09-19 23:16:07,2,0,1,0,0,1,0,23,2016-09-19,False,23:16:07,"[23:15:00,23:20:00)"


In [288]:
pd_volume_train['time'] =  pd.to_datetime(pd_volume_train['time'] , format='%Y-%m-%d %H:%M:%S')
#pd_volume_train = pd_volume_train.set_index(['time_window'])

# 车流量
pd_volume_train = pd_volume_train.groupby([pd.Grouper(freq='5T', key='time'), 'tollgate_id', 'direction', 'time_window', 'date', 'hour', 'holiday']).size()\
       .reset_index().rename(columns = {0:'volume'})

In [289]:
pd_volume_train.head()

,time,tollgate_id,direction,time_window,date,hour,holiday,volume
0,2016-09-19,1,0,"[00:00:00,00:05:00)",2016-09-19,0,False,1
1,2016-09-19,1,1,"[00:00:00,00:05:00)",2016-09-19,0,False,27
2,2016-09-19,2,0,"[00:00:00,00:05:00)",2016-09-19,0,False,2
3,2016-09-19,3,0,"[00:00:00,00:05:00)",2016-09-19,0,False,3
4,2016-09-19,3,1,"[00:00:00,00:05:00)",2016-09-19,0,False,49


In [290]:
pd_volume_train['weekday'] = pd_volume_train['time'].dt.dayofweek + 1

In [291]:
#Adicionando valor da janela de tempo (2x) anterior na janela de tempo atual
pd_volume_train["volume_anterior"] = pd_volume_train.groupby(['direction', 'tollgate_id'])["volume"].transform("shift")
#condition = ~(df.groupby("time_window")['time'].transform("shift") == df['time'])
#df.loc[ condition,"volume_anterior" ] = None
pd_volume_train["volume_anterior"] =pd_volume_train.groupby(['direction', 'tollgate_id'])["volume_anterior"].fillna(method="ffill").fillna(0)
pd_volume_train.head()

,time,tollgate_id,direction,time_window,date,hour,holiday,volume,weekday,volume_anterior
0,2016-09-19,1,0,"[00:00:00,00:05:00)",2016-09-19,0,False,1,1,0.0
1,2016-09-19,1,1,"[00:00:00,00:05:00)",2016-09-19,0,False,27,1,0.0
2,2016-09-19,2,0,"[00:00:00,00:05:00)",2016-09-19,0,False,2,1,0.0
3,2016-09-19,3,0,"[00:00:00,00:05:00)",2016-09-19,0,False,3,1,0.0
4,2016-09-19,3,1,"[00:00:00,00:05:00)",2016-09-19,0,False,49,1,0.0


In [293]:
#Adicionando valor da janela de tempo (2x) anterior na janela de tempo atual
pd_volume_train["volume_anterior_2"] = pd_volume_train.groupby(['direction', 'tollgate_id'])["volume"].transform("shift", 2)
#condition = ~(df.groupby("time_window")['time'].transform("shift") == df['time'])
#df.loc[ condition,"volume_anterior" ] = None
pd_volume_train["volume_anterior_2"] =pd_volume_train.groupby(['direction', 'tollgate_id'])["volume_anterior_2"].fillna(method="ffill").fillna(0)
pd_volume_train.head()

,time,tollgate_id,direction,time_window,date,hour,holiday,volume,weekday,volume_anterior,volume_anterior_2
0,2016-09-19,1,0,"[00:00:00,00:05:00)",2016-09-19,0,False,1,1,0.0,0.0
1,2016-09-19,1,1,"[00:00:00,00:05:00)",2016-09-19,0,False,27,1,0.0,0.0
2,2016-09-19,2,0,"[00:00:00,00:05:00)",2016-09-19,0,False,2,1,0.0,0.0
3,2016-09-19,3,0,"[00:00:00,00:05:00)",2016-09-19,0,False,3,1,0.0,0.0
4,2016-09-19,3,1,"[00:00:00,00:05:00)",2016-09-19,0,False,49,1,0.0,0.0


In [294]:
pd_volume_train['media_weekday'] = pd_volume_train.groupby(['time_window', 'weekday', 'direction', 'tollgate_id'])["volume"].transform(np.mean)

In [295]:
pd_volume_train['desvio_weekady'] = pd_volume_train.groupby(['time_window', 'weekday', 'direction', 'tollgate_id'])["volume"].transform(np.std)

In [296]:
pd_volume_train.isnull().sum()

time                   0
tollgate_id            0
direction              0
time_window            0
date                   0
hour                   0
holiday                0
volume                 0
weekday                0
volume_anterior        0
volume_anterior_2      0
media_weekday          0
desvio_weekady       154
dtype: int64

In [297]:
pd_volume_train['desvio_weekady'].fillna(pd_volume_train.groupby(['time_window', 'weekday', 'direction', 'tollgate_id'])["volume"].transform(np.mean), inplace=True)
pd_volume_train.isnull().sum()

time                 0
tollgate_id          0
direction            0
time_window          0
date                 0
hour                 0
holiday              0
volume               0
weekday              0
volume_anterior      0
volume_anterior_2    0
media_weekday        0
desvio_weekady       0
dtype: int64

In [338]:
pd_volume_train

,time,tollgate_id,direction,time_window,date,hour,holiday,volume,weekday,volume_anterior,volume_anterior_2,media_weekday,desvio_weekady,media_dia_hora,desvio_dia_hora
0,2016-09-19 00:00:00,1,0,"[00:00:00,00:05:00)",2016-09-19,0,False,1,1,0.0,0.0,3.333333,2.081666,1,1.0
1,2016-09-19 00:00:00,1,1,"[00:00:00,00:05:00)",2016-09-19,0,False,27,1,0.0,0.0,29.000000,14.212670,27,27.0
2,2016-09-19 00:00:00,2,0,"[00:00:00,00:05:00)",2016-09-19,0,False,2,1,0.0,0.0,1.500000,0.707107,2,2.0
3,2016-09-19 00:00:00,3,0,"[00:00:00,00:05:00)",2016-09-19,0,False,3,1,0.0,0.0,4.600000,1.816590,3,3.0
4,2016-09-19 00:00:00,3,1,"[00:00:00,00:05:00)",2016-09-19,0,False,49,1,0.0,0.0,35.600000,17.169741,49,49.0
5,2016-09-19 00:05:00,1,0,"[00:05:00,00:10:00)",2016-09-19,0,False,4,1,1.0,0.0,4.200000,1.788854,4,4.0
6,2016-09-19 00:05:00,1,1,"[00:05:00,00:10:00)",2016-09-19,0,False,43,1,27.0,0.0,29.800000,13.627179,43,43.0
7,2016-09-19 00:05:00,3,0,"[00:05:00,00:10:00)",2016-09-19,0,False,3,1,3.0,0.0,4.000000,2.345208,3,3.0
8,2016-09-19 00:05:00,3,1,"[00:05:00,00:10:00)",2016-09-19,0,False,52,1,49.0,0.0,30.800000,16.634302,52,52.0
9,2016-09-19 00:10:00,1,0,"[00:10:00,00:15:00)",2016-09-19,0,False,2,1,4.0,1.0,3.000000,1.732051,2,2.0


In [353]:
pd_volume_train['media_dia_hora'] = pd_volume_train.groupby(['date', 'hour', 'tollgate_id', 'direction'])["volume"].transform(np.mean)

In [354]:
pd_volume_train

,time,tollgate_id,direction,time_window,date,hour,holiday,volume,weekday,volume_anterior,volume_anterior_2,media_weekday,desvio_weekady,media_dia_hora,desvio_dia_hora
0,2016-09-19 00:00:00,1,0,"[00:00:00,00:05:00)",2016-09-19,0,False,1,1,0.0,0.0,3.333333,2.081666,2.333333,1.0
1,2016-09-19 00:00:00,1,1,"[00:00:00,00:05:00)",2016-09-19,0,False,27,1,0.0,0.0,29.000000,14.212670,18.916667,27.0
2,2016-09-19 00:00:00,2,0,"[00:00:00,00:05:00)",2016-09-19,0,False,2,1,0.0,0.0,1.500000,0.707107,2.000000,2.0
3,2016-09-19 00:00:00,3,0,"[00:00:00,00:05:00)",2016-09-19,0,False,3,1,0.0,0.0,4.600000,1.816590,4.083333,3.0
4,2016-09-19 00:00:00,3,1,"[00:00:00,00:05:00)",2016-09-19,0,False,49,1,0.0,0.0,35.600000,17.169741,32.166667,49.0
5,2016-09-19 00:05:00,1,0,"[00:05:00,00:10:00)",2016-09-19,0,False,4,1,1.0,0.0,4.200000,1.788854,2.333333,4.0
6,2016-09-19 00:05:00,1,1,"[00:05:00,00:10:00)",2016-09-19,0,False,43,1,27.0,0.0,29.800000,13.627179,18.916667,43.0
7,2016-09-19 00:05:00,3,0,"[00:05:00,00:10:00)",2016-09-19,0,False,3,1,3.0,0.0,4.000000,2.345208,4.083333,3.0
8,2016-09-19 00:05:00,3,1,"[00:05:00,00:10:00)",2016-09-19,0,False,52,1,49.0,0.0,30.800000,16.634302,32.166667,52.0
9,2016-09-19 00:10:00,1,0,"[00:10:00,00:15:00)",2016-09-19,0,False,2,1,4.0,1.0,3.000000,1.732051,2.333333,2.0


In [355]:
pd_volume_train['desvio_dia_hora'] = pd_volume_train.groupby(['date', 'hour', 'direction', 'tollgate_id'])["volume"].transform(np.std)


pd_volume_train['desvio_dia_hora'].fillna(pd_volume_train.groupby(['date', 'hour', 'direction', 'tollgate_id'])["volume"].transform(np.mean), inplace=True)
pd_volume_train.isnull().sum()

time                 0
tollgate_id          0
direction            0
time_window          0
date                 0
hour                 0
holiday              0
volume               0
weekday              0
volume_anterior      0
volume_anterior_2    0
media_weekday        0
desvio_weekady       0
media_dia_hora       0
desvio_dia_hora      0
dtype: int64

#SALVANDO ARQUIVO DE TREINO INICIANDO MESMAS MODIFICAÇÕES PARA ARQUIVO DE TESTE

In [356]:
pd_volume_train.to_csv('dados_treino_volume_5Min.csv', index = False)

In [367]:
pd_volume_test = pd.read_csv('processed_test_volume2.csv')
pd_volume_test.head()
del pd_volume_test['time_window']

In [368]:
pd_volume_test['time'] = pd.to_datetime(pd_volume_test['time'], format = '%Y-%m-%d %H:%M:%S')
pd_volume_test['t'] = pd_volume_test['time'].dt.time
pd_volume_test['time_window'] = pd_volume_test['t'].apply(get_timewindow20)
#del pd_volume_train['t']
pd_volume_test.tail()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type,weekday,weekend,hour,date,holiday,t,time_window
29436,2016-10-24 16:27:39,3,1,1,0,0,1,0,16,2016-10-24,False,16:27:39,"[16:20:00,16:40:00)"
29437,2016-10-24 16:27:18,3,1,1,0,0,1,0,16,2016-10-24,False,16:27:18,"[16:20:00,16:40:00)"
29438,2016-10-24 16:37:42,3,1,5,0,1,1,0,16,2016-10-24,False,16:37:42,"[16:20:00,16:40:00)"
29439,2016-10-24 16:47:48,3,1,5,0,1,1,0,16,2016-10-24,False,16:47:48,"[16:40:00,17:00:00)"
29440,2016-10-24 16:48:40,3,1,2,0,0,1,0,16,2016-10-24,False,16:48:40,"[16:40:00,17:00:00)"


In [369]:
pd_volume_test['time'] =  pd.to_datetime(pd_volume_test['time'] , format='%Y-%m-%d %H:%M:%S')
#pd_volume_train = pd_volume_train.set_index(['time_window'])

# 车流量
pd_volume_test = pd_volume_test.groupby([pd.Grouper(freq='20T', key='time'), 'tollgate_id', 'direction', 'time_window', 'date', 'hour', 'holiday']).size()\
       .reset_index().rename(columns = {0:'volume'})

In [370]:
pd_volume_test['weekday'] = pd_volume_test['time'].dt.dayofweek + 1

In [371]:
#Adicionando valor da janela de tempo (2x) anterior na janela de tempo atual
pd_volume_test["volume_anterior"] = pd_volume_test.groupby(['direction', 'tollgate_id'])["volume"].transform("shift")
#condition = ~(df.groupby("time_window")['time'].transform("shift") == df['time'])
#df.loc[ condition,"volume_anterior" ] = None
pd_volume_test["volume_anterior"] =pd_volume_test.groupby(['direction', 'tollgate_id'])["volume_anterior"].fillna(method="ffill").fillna(0)
pd_volume_test.head()

,time,tollgate_id,direction,time_window,date,hour,holiday,volume,weekday,volume_anterior
0,2016-10-18 06:00:00,1,0,"[06:00:00,06:20:00)",2016-10-18,6,False,13,2,0.0
1,2016-10-18 06:00:00,1,1,"[06:00:00,06:20:00)",2016-10-18,6,False,37,2,0.0
2,2016-10-18 06:00:00,2,0,"[06:00:00,06:20:00)",2016-10-18,6,False,24,2,0.0
3,2016-10-18 06:00:00,3,0,"[06:00:00,06:20:00)",2016-10-18,6,False,30,2,0.0
4,2016-10-18 06:00:00,3,1,"[06:00:00,06:20:00)",2016-10-18,6,False,23,2,0.0


In [372]:
#Adicionando valor da janela de tempo (2x) anterior na janela de tempo atual
pd_volume_test["volume_anterior_2"] = pd_volume_test.groupby(['direction', 'tollgate_id'])["volume"].transform("shift", 2)
#condition = ~(df.groupby("time_window")['time'].transform("shift") == df['time'])
#df.loc[ condition,"volume_anterior" ] = None
pd_volume_test["volume_anterior_2"] =pd_volume_test.groupby(['direction', 'tollgate_id'])["volume_anterior_2"].fillna(method="ffill").fillna(0)
pd_volume_test.head()

,time,tollgate_id,direction,time_window,date,hour,holiday,volume,weekday,volume_anterior,volume_anterior_2
0,2016-10-18 06:00:00,1,0,"[06:00:00,06:20:00)",2016-10-18,6,False,13,2,0.0,0.0
1,2016-10-18 06:00:00,1,1,"[06:00:00,06:20:00)",2016-10-18,6,False,37,2,0.0,0.0
2,2016-10-18 06:00:00,2,0,"[06:00:00,06:20:00)",2016-10-18,6,False,24,2,0.0,0.0
3,2016-10-18 06:00:00,3,0,"[06:00:00,06:20:00)",2016-10-18,6,False,30,2,0.0,0.0
4,2016-10-18 06:00:00,3,1,"[06:00:00,06:20:00)",2016-10-18,6,False,23,2,0.0,0.0


In [373]:
pd_volume_test['media_weekday'] = pd_volume_test.groupby(['time_window', 'direction', 'tollgate_id'])["volume"].transform(np.mean)

In [374]:
pd_volume_test['desvio_weekady'] = pd_volume_test.groupby(['time_window', 'direction', 'tollgate_id'])["volume"].transform(np.std)

In [375]:
pd_volume_test.tail()

,time,tollgate_id,direction,time_window,date,hour,holiday,volume,weekday,volume_anterior,volume_anterior_2,media_weekday,desvio_weekady
415,2016-10-24 16:40:00,1,0,"[16:40:00,17:00:00)",2016-10-24,16,False,44,1,50.0,49.0,50.571429,12.190941
416,2016-10-24 16:40:00,1,1,"[16:40:00,17:00:00)",2016-10-24,16,False,103,1,108.0,99.0,102.285714,14.795430
417,2016-10-24 16:40:00,2,0,"[16:40:00,17:00:00)",2016-10-24,16,False,80,1,75.0,69.0,72.857143,8.355209
418,2016-10-24 16:40:00,3,0,"[16:40:00,17:00:00)",2016-10-24,16,False,130,1,100.0,115.0,125.428571,24.636404
419,2016-10-24 16:40:00,3,1,"[16:40:00,17:00:00)",2016-10-24,16,False,81,1,80.0,101.0,88.857143,11.186727


In [376]:
#pd_volume_train['desvio_weekady'].fillna(pd_volume_train.groupby(['time_window', 'weekday', 'direction', 'tollgate_id'])["volume"].transform(np.mean), inplace=True)
pd_volume_test.isnull().sum()

time                 0
tollgate_id          0
direction            0
time_window          0
date                 0
hour                 0
holiday              0
volume               0
weekday              0
volume_anterior      0
volume_anterior_2    0
media_weekday        0
desvio_weekady       0
dtype: int64

In [377]:
pd_volume_test['media_dia_hora'] = pd_volume_test.groupby(['date', 'hour', 'direction', 'tollgate_id'])["volume"].transform(np.mean)

In [378]:
pd_volume_test['desvio_dia_hora'] = pd_volume_test.groupby(['date', 'hour', 'direction', 'tollgate_id'])["volume"].transform(np.std)


pd_volume_test['desvio_dia_hora'].fillna(pd_volume_test.groupby(['date', 'hour', 'direction', 'tollgate_id'])["volume"].transform(np.mean), inplace=True)
pd_volume_test.isnull().sum()

time                 0
tollgate_id          0
direction            0
time_window          0
date                 0
hour                 0
holiday              0
volume               0
weekday              0
volume_anterior      0
volume_anterior_2    0
media_weekday        0
desvio_weekady       0
media_dia_hora       0
desvio_dia_hora      0
dtype: int64

In [379]:
pd_volume_test.to_csv('dados_teste_volume_5Min.csv', index = False)

In [390]:
def feature_format():
    v_train = pd.read_csv('dados_treino_volume_5Min.csv')
    v_test = pd.read_csv('dados_teste_volume_5Min.csv')
    #v_train = v_train.set_index(['time'])
    #v_test = v_test.set_index(['time'])
    #volume_train = v_train.groupby(['time_window','tollgate_id','direction','date', 'hour', 'has_etc', 'holiday']).size().reset_index().rename(columns = {0:'volume'})
    #volume_test = v_test.groupby(['time_window','tollgate_id','direction','date', 'hour', 'has_etc', 'holiday']).size().reset_index().rename(columns = {0:'volume'})
    #print(volume_train)                
    x = pd.Series(v_train['time_window'].unique())
    xT = pd.Series(v_test['time_window'].unique())
    s = pd.Series(range(len(x)),index = x.values)
    sT = pd.Series(range(len(xT)),index = xT.values)
    v_train['window_n'] = v_train['time_window'].map(s)
    v_test['window_n'] = v_test['time_window'].map(sT)
#        print vol_test.tail()
    #volume_train['weekday'] = v_train['weekday']
    #volume_test['weekday'] = v_test['weekday']
    
    feature_train = v_train.drop('volume', axis = 1)
    feature_test = v_test.drop('volume',axis = 1)
    values_train = v_train['volume'].values
    values_test = v_test['volume'].values
    
    return feature_train, feature_test, values_train, values_test

In [391]:
feature_train, feature_test, values_train, values_test = feature_format()

In [399]:
values_test

array([ 13,  37,  24,  30,  23,  17,  47,  36,  42,  29,  21,  72,  46,
        85,  50,  31,  68,  83, 114,  56,  28,  94,  87, 131,  77,  47,
       105,  97, 164,  91,  52,  72,  74, 116,  81,  38, 102,  78,  87,
        91,  35, 116,  53, 106,  83,  57, 102,  62, 109,  84,  45,  84,
        63, 117,  92,  53, 113,  83, 118,  83,  12,  48,  26,  24,  21,
        16,  28,  45,  57,  26,  17,  57,  47,  75,  43,  22,  67,  87,
       119,  53,  38,  98,  95, 137,  63,  41, 103,  99, 152, 104,  40,
        83,  70, 107,  99,  34,  99,  69,  96, 108,  50,  92,  58,  95,
        79,  39,  82,  55, 109,  82,  40,  80,  61, 105,  89,  36,  85,
        64,  95,  75,   8,  46,  21,  37,  16,  16,  27,  23,  44,  49,
        23,  55,  42,  73,  37,  22,  71,  76, 114,  47,  33,  88,  89,
       123,  60,  37, 111, 112, 120, 104,  40,  97,  72,  92, 100,  41,
       115,  79, 107, 105,  42, 120,  70, 108,  74,  46,  90,  66, 121,
        92,  36,  73,  67, 106,  84,  37, 119,  75, 147,  88,   

In [416]:
rng = np.random.RandomState(1)
regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 100),
                         n_estimators=50, random_state = rng)

In [415]:
regr.fit(feature_train[['window_n','tollgate_id', 'direction','media_weekday','weekday', 'desvio_weekady', 'media_dia_hora', 'desvio_dia_hora', 'volume_anterior']], values_train)

y_pred = regr.predict(feature_test[['window_n','tollgate_id', 'direction', 'media_weekday', 'weekday', 'desvio_weekady', 'media_dia_hora', 'desvio_dia_hora', 'volume_anterior']])

mape = np.mean(np.abs((y_pred - values_test)/values_test))
mape

0.3068713323346971

In [333]:
#Função que calcula o MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [334]:
mean_absolute_percentage_error(y_pred, values_test)

36.23203872476894

In [417]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1500, random_state = 42)
# Train the model on training data
rf.fit(feature_train[['window_n','tollgate_id', 'direction', 'media_weekday', 'weekday', 'desvio_weekady', 'media_dia_hora', 'desvio_dia_hora', 'volume_anterior']], values_train);

In [418]:
# Use the forest's predict method on the test data
predictions = rf.predict(feature_test[['window_n','tollgate_id', 'direction', 'media_weekday', 'weekday', 'desvio_weekady', 'media_dia_hora', 'desvio_dia_hora', 'volume_anterior']])
# Calculate the absolute errors
errors = abs(predictions - values_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 23.23 degrees.


In [419]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / values_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 68.88 %.


In [407]:
rmse = sqrt(mean_squared_error(values_test, y_pred))
rmse

38.03126908807003